<a href="https://colab.research.google.com/github/serhatyagci/FASTAPI-Stok_giris_cikis/blob/main/stok_giris_cikis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gerekli kütüphanelerin kurulması.

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok sqlalchemy databases fastapi

FastAPI, SQLAlchemy ve diğer kütüphanelerin import edilmesi:

In [ ]:
from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime
from sqlalchemy.orm import sessionmaker, relationship, Session
from sqlalchemy.orm import declarative_base
from datetime import datetime
from typing import List

Fast api uygulamasının başlatılması

In [ ]:
app = FastAPI()

Veritabanı oluşturulması

In [ ]:
# SQLite veritabanı URL'si
SQLALCHEMY_DATABASE_URL = "sqlite:///./test.db"

# Veritabanı bağlantısı ve session yapılandırması
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Veritabanı tabanını tanımlanması
Base = declarative_base()

# Stoklar Tablosu
class Stok(Base):
    __tablename__ = "stoklar"
    stok_id = Column(Integer, primary_key=True, index=True)
    stok_kod = Column(String, index=True)
    stok_adi = Column(String)
    stok_barkod = Column(String)
    stok_fiyat = Column(Float)

    #veritabanı ilişkilerinin kurulması
    alis = relationship("Alis", back_populates="stok")
    satis = relationship("Satis", back_populates="stok")

# Kasiyer Tablosu
class Kasiyer(Base):
    __tablename__ = "kasiyer"
    kasiyer_id = Column(Integer, primary_key=True, index=True)
    kasiyer_kod = Column(String, index=True)
    kasiyer_adi = Column(String)
    cinsiyet = Column(String)

    #veritabanı ilişkilerinin kurulması
    alis = relationship("Alis", back_populates="kasiyer")
    satis = relationship("Satis", back_populates="kasiyer")

# Alis Tablosu
class Alis(Base):
    __tablename__ = "alis"
    alis_id = Column(Integer, primary_key=True, index=True)
    stok_id = Column(Integer, ForeignKey("stoklar.stok_id"))
    alis_adet = Column(Integer)
    kasiyer_id = Column(Integer, ForeignKey("kasiyer.kasiyer_id"))
    alis_tarihi = Column(DateTime, default=datetime.utcnow)

    #veritabanı ilişkilerinin kurulması
    stok = relationship("Stok", back_populates="alis")
    kasiyer = relationship("Kasiyer", back_populates="alis")

# Satis Tablosu
class Satis(Base):
    __tablename__ = "satis"
    satis_id = Column(Integer, primary_key=True, index=True)
    stok_id = Column(Integer, ForeignKey("stoklar.stok_id"))
    satis_adet = Column(Integer)
    kasiyer_id = Column(Integer, ForeignKey("kasiyer.kasiyer_id"))
    satis_tarihi = Column(DateTime, default=datetime.utcnow)

    #veritabanı ilişkilerinin kurulması
    stok = relationship("Stok", back_populates="satis")
    kasiyer = relationship("Kasiyer", back_populates="satis")

# Veritabanı oluşturma (Tabloları oluşturur)
Base.metadata.create_all(bind=engine)

Modellerin oluşturulması(API'ler için)

In [ ]:
# Pydantic modelleri: gelen api istek verilerinin doğruluğunu kontrol etmek için kullanılır.
class StokCreate(BaseModel):
    stok_kod: str
    stok_adi: str
    stok_barkod: str
    stok_fiyat: float

    class Config:
        orm_mode = True

class KasiyerCreate(BaseModel):
    kasiyer_kod: str
    kasiyer_adi: str
    cinsiyet: str

    class Config:
        orm_mode = True

class AlisCreate(BaseModel):
    stok_id: int
    alis_adet: int
    kasiyer_id: int

    class Config:
        orm_mode = True

class SatisCreate(BaseModel):
    stok_id: int
    satis_adet: int
    kasiyer_id: int

    class Config:
        orm_mode = True

Her çağırılan api için veritabanı bağlantısı sağlamak için oluşturulan session ve kullandıktan sonra kapanır.

In [ ]:
# Veritabanı session yönetimi
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

CRUD işlemleri ve API endpointleri

In [ ]:
# Stok ekleme
@app.post("/stoklar", response_model=StokCreate)
def create_stok(stok: StokCreate, db: Session = Depends(get_db)):
    db_stok = Stok(stok_kod=stok.stok_kod, stok_adi=stok.stok_adi, stok_barkod=stok.stok_barkod, stok_fiyat=stok.stok_fiyat)
    db.add(db_stok)
    db.commit()
    db.refresh(db_stok)
    return db_stok

# Kasiyer ekleme
@app.post("/kasiyer", response_model=KasiyerCreate)
def create_kasiyer(kasiyer: KasiyerCreate, db: Session = Depends(get_db)):
    db_kasiyer = Kasiyer(kasiyer_kod=kasiyer.kasiyer_kod, kasiyer_adi=kasiyer.kasiyer_adi, cinsiyet=kasiyer.cinsiyet)
    db.add(db_kasiyer)
    db.commit()
    db.refresh(db_kasiyer)
    return db_kasiyer

# Alis ekleme
@app.post("/alis", response_model=AlisCreate)
def create_alis(alis: AlisCreate, db: Session = Depends(get_db)):
    db_alis = Alis(stok_id=alis.stok_id, alis_adet=alis.alis_adet, kasiyer_id=alis.kasiyer_id)
    db.add(db_alis)
    db.commit()
    db.refresh(db_alis)
    return db_alis

# Satis ekleme
@app.post("/satis", response_model=SatisCreate)
def create_satis(satis: SatisCreate, db: Session = Depends(get_db)):
    db_satis = Satis(stok_id=satis.stok_id, satis_adet=satis.satis_adet, kasiyer_id=satis.kasiyer_id)
    db.add(db_satis)
    db.commit()
    db.refresh(db_satis)
    return db_satis

# Stok güncelleme
@app.put("/stoklar/{stok_id}", response_model=StokCreate)
def update_stok(stok_id: int, stok: StokCreate, db: Session = Depends(get_db)):
    db_stok = db.query(Stok).filter(Stok.stok_id == stok_id).first()
    if not db_stok:
        raise HTTPException(status_code=404, detail="Stok not found")

    db_stok.stok_kod = stok.stok_kod if stok.stok_kod else db_stok.stok_kod
    db_stok.stok_adi = stok.stok_adi if stok.stok_adi else db_stok.stok_adi
    db_stok.stok_barkod = stok.stok_barkod if stok.stok_barkod else db_stok.stok_barkod
    db_stok.stok_fiyat = stok.stok_fiyat if stok.stok_fiyat else db_stok.stok_fiyat

    db.commit()
    db.refresh(db_stok)
    return db_stok

# Kasiyer güncelleme
@app.put("/kasiyer/{kasiyer_id}", response_model=KasiyerCreate)
def update_kasiyer(kasiyer_id: int, kasiyer: KasiyerCreate, db: Session = Depends(get_db)):
    db_kasiyer = db.query(Kasiyer).filter(Kasiyer.kasiyer_id == kasiyer_id).first()
    if not db_kasiyer:
        raise HTTPException(status_code=404, detail="Kasiyer not found")

    db_kasiyer.kasiyer_kod = kasiyer.kasiyer_kod if kasiyer.kasiyer_kod else db_kasiyer.kasiyer_kod
    db_kasiyer.kasiyer_adi = kasiyer.kasiyer_adi if kasiyer.kasiyer_adi else db_kasiyer.kasiyer_adi
    db_kasiyer.cinsiyet = kasiyer.cinsiyet if kasiyer.cinsiyet else db_kasiyer.cinsiyet

    db.commit()
    db.refresh(db_kasiyer)
    return db_kasiyer

# Stok silme
@app.delete("/stoklar/{stok_id}")
def delete_stok(stok_id: int, db: Session = Depends(get_db)):
    db_stok = db.query(Stok).filter(Stok.stok_id == stok_id).first()
    if db_stok:
        db.delete(db_stok)
        db.commit()
        return {"message": "Stok deleted successfully"}
    else:
        raise HTTPException(status_code=404, detail="Stok not found")

# Kasiyer silme
@app.delete("/kasiyer/{kasiyer_id}")
def delete_kasiyer(kasiyer_id: int, db: Session = Depends(get_db)):
    db_kasiyer = db.query(Kasiyer).filter(Kasiyer.kasiyer_id == kasiyer_id).first()
    if db_kasiyer:
        db.delete(db_kasiyer)
        db.commit()
        return {"message": "Kasiyer deleted successfully"}
    else:
        raise HTTPException(status_code=404, detail="Kasiyer not found")
# Stoklar Tablosu - Tüm stokları listele
@app.get("/stoklar", response_model=List[StokCreate])
def get_stoklar(db: Session = Depends(get_db)):
    db_stoklar = db.query(Stok).all()
    return db_stoklar

# Kasiyerler Tablosu - Tüm kasiyerleri listele
@app.get("/kasiyer", response_model=List[KasiyerCreate])
def get_kasiyerler(db: Session = Depends(get_db)):
    db_kasiyerler = db.query(Kasiyer).all()
    return db_kasiyerler

# Alis Tablosu - Tüm alışları listele
@app.get("/alis", response_model=List[AlisCreate])
def get_alislar(db: Session = Depends(get_db)):
    db_alislar = db.query(Alis).all()
    return db_alislar

# Satis Tablosu - Tüm satışları listele
@app.get("/satis", response_model=List[SatisCreate])
def get_satislar(db: Session = Depends(get_db)):
    db_satislar = db.query(Satis).all()
    return db_satislar

# Stok Durum Raporu - Stok id'lere göre giriş ve çıkışları toplar, kalan tutarı hesaplar
from sqlalchemy import func  # func fonksiyonunu import et

@app.get("/stok-durum-raporu")
def get_stok_durum_raporu(stok_id: int, db: Session = Depends(get_db)):
    try:
        # Veritabanı sorgusu ile belirli stok_id'ye göre alınan ve satılan miktarları hesapla
        stok_durum = db.query(
            Stok.stok_id,
            Stok.stok_adi,
            func.sum(Alis.alis_adet).label('toplam_alis'),
            func.sum(Satis.satis_adet).label('toplam_satis')
        ).join(Alis, Alis.stok_id == Stok.stok_id, isouter=True).join(Satis, Satis.stok_id == Stok.stok_id, isouter=True) \
          .filter(Stok.stok_id == stok_id) \
          .group_by(Stok.stok_id).first()

        if not stok_durum:
            raise HTTPException(status_code=404, detail="Stok bulunamadı")

        # Sonuçları hesapla
        stok_durum_result = {
            "stok_id": stok_durum.stok_id,
            "stok_adi": stok_durum.stok_adi,
            "toplam_alis": stok_durum.toplam_alis or 0,
            "toplam_satis": stok_durum.toplam_satis or 0,
            "kalan_stok": (stok_durum.toplam_alis or 0) - (stok_durum.toplam_satis or 0)
        }

        return {"stok_durum": stok_durum_result}

    except Exception as e:
        # Hata durumunda 500 döndür
        return {"error": str(e)}, 500

# Kasiyer Satış Raporu - Tüm kasiyerlerin toplam sattığı satışlar toplamı
@app.get("/kasiyer-alis-satis-raporu")
def get_kasiyer_alis_satis_raporu(kasiyer_id: int, db: Session = Depends(get_db)):
    try:
        # Veritabanı sorgusu ile belirli bir kasiyerin yaptığı toplam satışları ve girişleri hesapla
        kasiyer_satis = db.query(
            Kasiyer.kasiyer_id,
            Kasiyer.kasiyer_adi,
            func.sum(Satis.satis_adet).label('toplam_satis'),
            func.sum(Alis.alis_adet).label('toplam_alis')
        ).join(Satis, Satis.kasiyer_id == Kasiyer.kasiyer_id) \
          .join(Alis, Alis.kasiyer_id == Kasiyer.kasiyer_id) \
          .filter(Kasiyer.kasiyer_id == kasiyer_id) \
          .group_by(Kasiyer.kasiyer_id).first()




        if not kasiyer_satis and not kasiyer_giris:
            raise HTTPException(status_code=404, detail="Kasiyer bulunamadı")

        # Sonuçları hesapla
        kasiyer_satis_result = {
            "kasiyer_id": kasiyer_satis.kasiyer_id,
            "kasiyer_adi": kasiyer_satis.kasiyer_adi,
            "toplam_satis": kasiyer_satis.toplam_satis or 0,
            "toplam_giris": kasiyer_satis.toplam_alis or 0
        }

        return {"kasiyer_satis": kasiyer_satis_result}

    except Exception as e:
        # Hata durumunda 500 döndür
        return {"error": str(e)}, 500

Ngrok'u kullanabilmek için gereken token

In [ ]:
!ngrok config add-authtoken 2qWoHdQu8GJmLe7D2fOgfUcj7KS_BLzF5GwQXAz1JYakQ8o

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


FastAPI uygulamamı çalıştırmak ve bir ngrok tüneli ile internet üzerinden erişilebilir hale getirmek için

In [ ]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Colab'da asyncio'yu tekrar başlatmaya izin ver
nest_asyncio.apply()

# Sunucuya bir ngrok tüneli bağlayın
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# FastAPI uygulamasını başlat
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://2784-34-106-24-185.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [146]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     159.146.117.151:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     159.146.117.151:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     159.146.117.151:0 - "GET /kasiyer-alis-satis-raporu?kasiyer_id=1 HTTP/1.1" 200 OK
INFO:     159.146.117.151:0 - "GET /stoklar HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [146]
